## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [1]:
import random
import pandas as pd
import numpy as np

In [2]:
A = [random.randrange(0, 1000) for _ in range(1000)]

B = [x+100 for x in A]

print(sum(B)/len(B))

606.331


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [3]:
from pandas._testing import rands_array

dftable = pd.DataFrame(np.random.randint(0,100,size=(2_000_000, 4)), columns=list('ABCD'))
dftable['key'] = rands_array(10, 2_000_000)
engelsk = dftable[dftable.key.str.startswith('ABCDI')]
engelsk

,A,B,C,D,key


## Лабораторная работа 3

In [4]:
import line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [5]:
recipes = pd.read_csv('data/recipes_sample.csv', dtype={'n_steps': 'Int32', 'n_ingredients': 'Int32'})  
reviews = pd.read_csv('data/reviews_sample.csv')
reviews.info()
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126696 entries, 0 to 126695
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  126696 non-null  int64 
 1   user_id     126696 non-null  int64 
 2   recipe_id   126696 non-null  int64 
 3   date        126696 non-null  object
 4   rating      126696 non-null  int64 
 5   review      126679 non-null  object
dtypes: int64(4), object(2)
memory usage: 5.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            30000 non-null  object
 1   id              30000 non-null  int64 
 2   minutes         30000 non-null  int64 
 3   contributor_id  30000 non-null  int64 
 4   submitted       30000 non-null  object
 5   n_steps         18810 non-null  Int32 
 6   description     29377 non-null  object
 7   n_i

In [6]:
reviews.date = pd.to_datetime(reviews.date)
reviews.rename( columns={'Unnamed: 0':'id'}, inplace=True )
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126696 entries, 0 to 126695
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   id         126696 non-null  int64         
 1   user_id    126696 non-null  int64         
 2   recipe_id  126696 non-null  int64         
 3   date       126696 non-null  datetime64[ns]
 4   rating     126696 non-null  int64         
 5   review     126679 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 5.8+ MB


In [7]:
ratings = [val.rating for _, val in reviews[reviews.date.dt.year == 2010].iterrows()]
# for indx, val in reviews[reviews.date.dt.year == 2010].iterrows():
# 	print('indx: ', indx,'val: ',  val)
sum(ratings)/len(ratings)


4.4544402182900615

In [8]:
import time

def Aitter():
	ratings = [val.rating for _, val in reviews[reviews.date.dt.year == 2010].iterrows()]
	return sum(ratings)/len(ratings)

def Bitter():
	global filtered_table
	ratings = [val.rating for _, val in filtered_table.iterrows()]
	return sum(ratings)/len(ratings)

def Cseries_mean():
	return reviews[reviews.date.dt.year == 2010].rating.mean()

st = time.time()
res = Aitter()
et = time.time()
print('A: itterrows for whole table\n\n\tres: ', res, '\n\ttime:', et - st )


filtered_table = reviews[reviews.date.dt.year == 2010]

st = time.time()
res = Bitter()
et = time.time()
print('\n\nB: itterrows for filttered table\n\n\tres: ', res, '\n\ttime:', et - st )

st = time.time()
res = Cseries_mean()
et = time.time()
print('\n\nC: Series.mean\n\n\tres: ', res, '\n\ttime:', et - st )

A: itterrows for whole table

	res:  4.4544402182900615 
	time: 0.6279904842376709


B: itterrows for filttered table

	res:  4.4544402182900615 
	time: 0.5489821434020996


C: Series.mean

	res:  4.4544402182900615 
	time: 0.012997627258300781


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [9]:
%load_ext line_profiler
%lprun -f Aitter Aitter()

Timer unit: 1e-07 s

Total time: 2.71745 s
File: C:\Users\tomas\AppData\Local\Temp\ipykernel_17544\3975581674.py
Function: Aitter at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def Aitter():
     4         1   27173289.0 27173289.0    100.0  	ratings = [val.rating for _, val in reviews[reviews.date.dt.year == 2010].iterrows()]
     5         1       1231.0   1231.0      0.0  	return sum(ratings)/len(ratings)

In [10]:
%lprun -f Bitter Bitter()

Timer unit: 1e-07 s

Total time: 2.91548 s
File: C:\Users\tomas\AppData\Local\Temp\ipykernel_17544\3975581674.py
Function: Bitter at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def Bitter():
     8                                           	global filtered_table
     9         1   29152964.0 29152964.0    100.0  	ratings = [val.rating for _, val in filtered_table.iterrows()]
    10         1       1808.0   1808.0      0.0  	return sum(ratings)/len(ratings)

In [11]:
%lprun -f Cseries_mean Cseries_mean()

Timer unit: 1e-07 s

Total time: 0.0168723 s
File: C:\Users\tomas\AppData\Local\Temp\ipykernel_17544\3975581674.py
Function: Cseries_mean at line 12

Line #      Hits         Time  Per Hit   % Time  Line Contents
    12                                           def Cseries_mean():
    13         1     168723.0 168723.0    100.0  	return reviews[reviews.date.dt.year == 2010].rating.mean()

In [12]:
def Bitter_improved():
	global filtered_table
	return filtered_table.rating.sum()/len(filtered_table)

Bitter_improved()

4.4544402182900615

In [13]:
%lprun -f Bitter_improved Bitter_improved()

Timer unit: 1e-07 s

Total time: 0.0006938 s
File: C:\Users\tomas\AppData\Local\Temp\ipykernel_17544\4008495202.py
Function: Bitter_improved at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Bitter_improved():
     2                                           	global filtered_table
     3         1       6938.0   6938.0    100.0  	return filtered_table.rating.sum()/len(filtered_table)

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [14]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():          # цикл for не оптимален, может быть заменен генератором
        recipe_id, review = row['recipe_id'], row['review']         # .iterrows так же является неоптимальным, для подсчета 
        words = review.split(' ')                                   # слов из одной колонки можно обращаться к ней напрямую
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [15]:
get_word_reviews_count(reviews)

{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [16]:
#too long
#%lprun -f get_word_reviews_count get_word_reviews_count(reviews) 

In [17]:
def get_word_reviews_count_imporved(reviews):
	# review_c = reviews.review.replace(r'[^\w\s]', '', regex=True).replace(r'\s+', ' ', regex=True)
	kek = pd.DataFrame(' '.join(recipes_c.tolist()).split())
	return kek.value_counts()

%lprun -f get_word_reviews_count_imporved get_word_reviews_count_imporved(reviews) 

NameError: name 'recipes_c' is not defined

In [ ]:
get_word_reviews_count_imporved(reviews) 

I                     291242
the                   266577
and                   219275
a                     166485
it                    133651
                       ...  
disposablereusable         1
disposing                  1
disposition                1
disppointed                1
ïŠ                         1
Length: 90065, dtype: int64

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [ ]:
import numba

non_zero_rev = reviews[reviews.rating > 0][['recipe_id', 'rating']]

def MAPE(raitings):
	ratings = [non_zero_rev[non_zero_rev['recipe_id'] == x]['rating'] for x in test_sample]
	avg_rating = sum(ratings) / len(ratings)
	mape = 0
	for rating in ratings:
		mape += abs((rating - avg_rating) / avg_rating)
	return mape / len(ratings) * 100

@numba.jit(nopython=True)
def MAPE_numba(raitings):
	avg_rating = sum(ratings) / len(ratings)
	mape = 0
	for rating in ratings:
		mape += abs((rating - avg_rating) / avg_rating)
	return mape / len(ratings) * 100

def MAPE_np(raitings):
	avg_rating = np.mean(ratings)
	mape = np.mean(np.abs((ratings - avg_rating) / avg_rating)) * 100
	return mape

@numba.jit(nopython=True)
def MAPE_np_numba(raitings):
	avg_rating = np.mean(ratings)
	mape = np.mean(np.abs((ratings - avg_rating) / avg_rating)) * 100
	return mape


In [ ]:

def MAPE_np(raitings):
	avg_rating = np.mean(ratings)
	mape = np.mean(np.abs((ratings - avg_rating) / avg_rating)) * 100
	return mape

In [ ]:
test_sample = non_zero_rev[:100].recipe_id
test_sample
# MAPE_numba

# ratings_np = [non_zero_rev[non_zero_rev['recipe_id'] == x]['rating'].to_numpy() for x in test_sample]
# ratings = [non_zero_rev[non_zero_rev['recipe_id'] == x]['rating'] for x in test_sample]

# results = pd.DataFrame([[MAPE(c) for c in ratings],
# 						[MAPE_np(c) for c in ratings_np],
# 						[MAPE_np_numba(c) for c in ratings_np],
# 						], columns =['MAPE', 'MAPE_np', 'MAPE_np_numba'])

def mape(y_test, pred):
  # Initializing the sum of errors
  error_sum = 0
  # Looping through the actual and predicted values
  for i in range(len(y_test)):
    # Calculating the absolute percentage error for each pair
    error = abs((y_test[i] - pred[i]) / y_test[i])
    # Adding the error to the sum
    error_sum += error
  # Calculating the mean of the errors
  mape = error_sum / len(y_test)
  return mape


# Calculating the average rating for each recipe
df = non_zero_rev.copy()
avg_rating = df.groupby("recipe_id")["rating"].mean()

# Merging the average rating with the original dataframe
df = df.merge(avg_rating, on="recipe_id", suffixes=("", "_avg"))

# Calling the MAPE function with numpy and numba
mape_score = mape(df["rating"], df["rating_avg"])


# df['mape'] = mape(df["rating"], df["rating_avg"])

#sum
rating_sums = df.groupby("recipe_id")["rating"].sum()

#lens
rating_counts = df.groupby("recipe_id")["rating"].count()

display(rating_sums)
display(rating_counts)

recipe_id
48         2
55        19
66        89
91        19
94        20
          ..
536360     5
536473     5
536547     5
536728     4
536729    19
Name: rating, Length: 27440, dtype: int64

recipe_id
48         1
55         4
66        18
91         4
94         4
          ..
536360     1
536473     1
536547     1
536728     1
536729     4
Name: rating, Length: 27440, dtype: int64

,recipe_id,rating,rating_avg
0,57993,5,4.818182
1,57993,5,4.818182
2,57993,5,4.818182
3,57993,5,4.818182
4,57993,5,4.818182
...,...,...,...
119886,94096,5,5.000000
119887,368208,2,2.000000
119888,154964,5,5.000000
119889,187418,5,5.000000


In [ ]:
dat = [list(x) for x in df.groupby("recipe_id")["rating"]]
dat

[[48,
  119214    2
  Name: rating, dtype: int64],
 [55,
  40087    5
  40088    5
  40089    4
  40090    5
  Name: rating, dtype: int64],
 [66,
  49524    5
  49525    5
  49526    5
  49527    5
  49528    4
  49529    5
  49530    5
  49531    5
  49532    5
  49533    5
  49534    5
  49535    5
  49536    5
  49537    5
  49538    5
  49539    5
  49540    5
  49541    5
  Name: rating, dtype: int64],
 [91,
  51606    5
  51607    5
  51608    4
  51609    5
  Name: rating, dtype: int64],
 [94,
  88098    5
  88099    5
  88100    5
  88101    5
  Name: rating, dtype: int64],
 [128,
  32301    5
  32302    5
  Name: rating, dtype: int64],
 [153,
  20445    5
  20446    5
  20447    5
  20448    4
  20449    5
  20450    5
  20451    5
  20452    5
  20453    5
  20454    5
  20455    4
  20456    5
  20457    5
  20458    5
  20459    5
  20460    5
  20461    5
  20462    5
  20463    5
  20464    5
  20465    5
  20466    5
  20467    5
  20468    5
  20469    5
  20470    5
  